In [ ]:
import numba
import numpy as np
from scipy import sparse

# Challenge 1: Text classification

In this problem, you'll be trying to visualize the seperation of text documents based on their contents. To do this, you'll build a graph where each node is a document, and has edges connecting to all other nodes weighted by their similarity. This is computationally expensive for a couple reasons. The first is that you're computing pairwise distances. This is something which can easily enought to do with numba, here's an example for calculating pairwise euclidean distances for an array:

In [ ]:
from sklearn.datasets import fetch_20newsgroups, make_blobs

In [ ]:
blobs_x, blobs_y = make_blobs(n_samples=100, n_features=10, centers=3)

In [ ]:
blobs_y

In [ ]:
@numba.njit
def pairwise(X, metric: "Callable"):
    dists = np.zeros((X.shape[0], X.shape[0]), dtype=np.float64)
    for i in range(X.shape[0]):
        for j in range(i+1, X.shape[0]):
            dists[i, j] = metric(X[i, :], X[j, :])
    return dists

In [ ]:
@numba.njit
def euclidean_dist(a, b):
    return np.sum((a - b) ** 2) ** 0.5

In [ ]:
%time d = pairwise(blobs_x, euclidean_dist)

However, both scipy and scikit-learn already have fast implementations for pairwise distances:

In [ ]:
import sklearn.metrics
import scipy.spatial

In [ ]:
%timeit sklearn.metrics.pairwise_distances(blobs_x, metric="euclidean")
%timeit scipy.spatial.distance.pdist(blobs_x, metric="euclidean")

So why do we need numba?

Well, text data is often represented as a matrix where the rows are documents and the columns are words. Since there are a lot of words used in a corpus, but not every document uses every word this matrix tends to have a lot of zeros. Another way to say this is that it's sparse. There are efficient representations for [sparse matrices](https://en.wikipedia.org/wiki/Sparse_matrix) included in scipy:

## [COO (COOrdinate) format](https://en.wikipedia.org/wiki/Sparse_matrix#Coordinate_list_(COO)) (`scipy.sparse.coo_matrix`)

Given a dense array like:

```
0 0 1 1 0
0 2 0 0 0
0 0 0 3 4
```

The coo sparse format stores the coordinates for each value and the value itself in three equal length arrays. The previous matrix would look like:

```
row  col  data
---  ---  ----
0    2    1
0    3    1
1    1    2
2    3    3
2    4    4

```


## [CSR (Compressed Sparse Row) format](https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_(CSR,_CRS_or_Yale_format)) (`scipy.sparse.csr_matrix`)

CSR format is similar to COO, except we've "compressed" the data along the rows. This replaces the row array with `indptr`, which instead of repeating the row number, just tells you ranges for which values of that row are defined. The matrix above would be represented as:

````
indptr indices data
------ ------- ----
0      2       1
2      3       1
3      1       2
5      3       3
       4       4
```

Note that `indices` and `data` are similar to COO, but `len(indptr) == (matrix.shape[0] + 1)`.

If we wanted to get all of the data points for row i, we could write:

```python
idx_slice = slice(matrix.indptr[i], matrix.indptr[i+1])
indices_i = matrix.indices[idx_slice]
data_i = matrix.data[idx_slice]
```

Or if we wanted to get indices for values of each row:

```python
for i in range(matrix.shape[0]):
    indices_i = matrix.indices[matrix.indptr[i]:matrix.indptr[i+1]]
# or
indices_by_row = np.split(matrix.indices, matrix.indptr[1:-1])
```

`CSR` format is how scikit learn typically represents text data. Lets take a look:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import make_pipeline

In [ ]:
preprocess = make_pipeline(
    CountVectorizer(),
    TfidfTransformer()
)

corpus = fetch_20newsgroups(categories=['talk.religion.misc', 'comp.graphics'])

In [ ]:
occurences = preprocess.fit_transform(corpus.data).astype(bool)
occurences

As you can see, we've encoded documents as a boolean sparse matrix, where `occurences[i, j] = True` if document `i` contains word `j`. Now, how should we tell which documents are closer to each other?

## Jaccard similarity

$$ Jaccard = \frac{|s_i \cap s_j|}{|s_i \cup s_j|} $$

So that the total number of words which show up in each article divided by the total number of unique words which show up in both. Or:

$$ Jaccard = \frac{\text{Number of shared words}}{\text{Total unique words}} $$

This can could be written in python like:

```python
def jaccard(a, b):
    a, b = set(a), set(b)
    len_intersect = a.intersection(b)
    return len_intersect / (len(a) + len(b) - len_intersect)
```

## The problem

### A.

`sklearn` doesn't have a method to calculate pairwise jaccard similarities for sparse matrices. Can you modify the pairwise code and jaccard similarity function to implement this using `numba`? A key part of this problem is figuring out which objects and functions will work with `numba`, and figuring out what values to pass into jitted functions.

In [ ]:
# Your code here

Once you're done with this, we can use `networkx` to visualize how the groups seperate by using the similarities as edge weights on a graph, the using a force directed layout

In [ ]:
import networkx as nx

In [ ]:
g = nx.from_numpy_array(result)
nx.draw_spring(g, node_color=corpus.target, s)

### B.

The implementation I gave for jaccard similarities is inefficient in this instance, since `set.intersection` is fairly slow. There's a faster way to calculate this.

*Hint*: Take advantage of the fact the csr matrix returned by sklearn has sorted indices for each row. That is:

```python
def issorted(x):
    return np.all(np.sort(x) == x)
assert all(map(issorted, np.split(occurences.indices, occurences.indptr[1:-1])))
```

# 2 DeJong attractors:

Based on an [Observable notebook from Mike Bostock](https://observablehq.com/@mbostock/making-webgl-dance?collection=@observablehq/webgl), let's visualize some attractors, mostly because they look cool: 

![dejong](dejong.png)

This problem is also a good example of parallelism. Notably, this problem parallelizes well on a GPU. If you have a CUDA or ROCm GPU I highly recommend you try and implement this function for that.

Make a numba-ized version of this function for the DeJong attractor. Once you have it working, try making it [run in parallel by passing `parallel=True`](http://numba.pydata.org/numba-doc/latest/user/parallel.html#numba-parallel) to `@jit`, this should give you a signifigant speed up.

In [ ]:
import numba
import numpy as np
from numpy import sin, cos
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [ ]:
def dejong(N, a, b, c, d, it=8):
    N = 500
    cur = np.zeros((N, N, 2), dtype="f8")
    cur[:, :, 0] = np.arange(N)[:, None] + np.zeros(N)
    cur[:, :, 1] = np.arange(N) + np.zeros(N)[:, None]
    prev = np.empty_like(cur)
    for i in range(it):
        prev[:] = cur[:]
        cur[:, :, 0] = sin(a * prev[:, :, 1]) - cos(b * prev[:, :, 0]) / 2
        cur[:, :, 1] = sin(c * prev[:, :, 0]) - cos(d * prev[:, :, 1]) / 2
    return cur

In [ ]:
out = dejong(100, -2, -2, 1.2, 2).reshape(-1, 2)

In [ ]:
plt.pcolor(np.histogram2d(out[:, 0], out[:, 1], 200)[0], cmap="Blues", norm=colors.LogNorm())

In [ ]:
# Your code here